## Libraries und Vairablen

In [15]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by  import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

In [16]:
buchhaus_new_last_30 = 'https://www.buchhaus.ch/de/heute/last30'

## Webscraping

/html/body/div[2]/div/div/main/div[2]/section[2]/div[1]/div[2]/div/span/span/span -> korrekt
/html/body/div[2]/div/div/main/div[2]/section[2]/div[1]/div[2]/div/span/span/span

In [17]:
PATH = "C:\Program Files (x86)\chromedriver.exe"
driver = webdriver.Chrome(PATH)
#load the page
driver.get(buchhaus_new_last_30)

# data frame
buch_df = pd.DataFrame(columns=['title',  
                            'preis',
                            'untertitel',
                            'autor', 
                            'verlag',
                            'sprache',
                            'einband',
                            'erscheinungsjahr',
                            'seiten',
                            'artikelnummer',
                            'isbn',
                            'auflage',
                            'reihe',
                            'reihenbandnummer',
                            'genre',
                            'text'])

''' 
Algorith explanation:
1. go to the i'th book and click on it
2. extract all the necessary data
3. add the data to the data frame
4. go back to main page
5. after 25 books go to the next page
'''

for page in range(1, 11):
    for book in range(1,26):
        time.sleep(2)
        try:
            buch = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/main/div[3]/div[2]/div[4]/div/div/div[{}]/div/div[1]/div/div/span/span/a'.format(book))
            buch.click()

            # get the data
            try: titel = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/main/div[2]/section[1]/div[2]/div[1]/h1/span/span').text
            except: titel = 'nan'

            try: preis = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/main/div[2]/section[1]/div[3]/div[1]/div/div/span[2]/span').text
            except: preis = 'nan'

            try: untertitel = driver.find_element(By.XPATH, '/html/body/div[3]/div[2]/form/div[1]/div[2]/div/div[2]/table/tbody/tr[1]/td[2]').text
            except: untertitel = 'nan'

            try: autor = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/main/div[2]/section[1]/div[2]/div[1]/div/div/span/span/span/span/a').text
            except: autor = 'nan'

            try: verlag = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/main/div[2]/section[2]/article[1]/div[2]/div[2]/div[1]/div[5]/span[2]/div/div/span/span/a').text
            except: verlag = 'nan'

            try: einband = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/main/div[2]/section[1]/div[2]/div[4]/span/span').text
            except: einband = 'nan'

            try: erscheinungsjahr = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/main/div[2]/section[2]/article[1]/div[2]/div[2]/div[1]/div[7]/span[2]/span').text
            except: erscheinungsjahr = 'nan'

            try: seiten = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/main/div[2]/section[2]/article[1]/div[2]/div[2]/div[1]/div[11]/span[2]/span').text
            except: seiten = 'nan'

            try: artikelnummer = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/main/div[2]/section[2]/article[1]/div[2]/div[2]/div[1]/div[12]/span[2]/span').text
            except: artikelnummer = 'nan'

            try: isbn = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/main/div[2]/section[2]/article[1]/div[2]/div[2]/div[1]/div[2]/span[2]/span').text
            except: isbn = 'nan'

            try: auflage = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/main/div[2]/section[2]/article[1]/div[2]/div[2]/div[1]/div[8]/span[2]/span').text
            except: auflage = 'nan'

            try: reihe = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/main/div[2]/section[2]/article[1]/div[2]/div[2]/div[1]/div[9]/span[2]/span/a').text
            except: reihe = 'nan'

            try: reihenbandnummer = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/main/div[2]/section[2]/article[1]/div[2]/div[2]/div[1]/div[10]/span[2]/span').text
            except: reihenbandnummer = 'nan'

            try: genre = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/main/div[2]/section[2]/article[1]/div[2]/div[2]/div[2]/div[2]/span[2]/span/a').text
            except: genre = 'nan'

            # get the 'Buchbeschreibungen'
            try: text = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/main/div[2]/section[2]/div[1]/div[2]/div/span/span/span').text
            except: text = 'nan'

            sprache = 'Deutsch'

        # Fehlermeldung
        except: 
            print('error mit Buch {} auf Seite {}'.format(book, page))
            continue
    

        df_temp = pd.DataFrame({'title': titel,  
                        'preis': preis,
                        'untertitel': untertitel,
                        'autor': autor, 
                        'verlag': verlag,
                        'sprache': sprache,
                        'einband': einband,
                        'erscheinungsjahr': erscheinungsjahr,
                        'seiten': seiten,
                        'artikelnummer':artikelnummer,
                        'isbn': isbn,
                        'auflage': auflage,
                        'reihe': reihe,
                        'reihenbandnummer': reihenbandnummer,
                        'genre': genre,
                        'text': text
                        }, index=[0])
        buch_df = pd.concat([buch_df, df_temp], ignore_index=True)
    
        #back to the main page
        driver.get(buchhaus_new_last_30)

    # go to the next page
    if page == 1: driver.find_element(By.XPATH, '/html/body/div[2]/div/div/main/div[3]/div[2]/div[1]/div[5]/div/a').click()
    else: driver.find_element(By.XPATH, '/html/body/div[2]/div/div/main/div[3]/div[2]/div[1]/div[5]/div/a[2]').click()
    time.sleep(1)


C:\Users\yanni\AppData\Local\Temp\ipykernel_14280\324257478.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)


error mit Buch 10 auf Seite 1
error mit Buch 11 auf Seite 1
error mit Buch 12 auf Seite 1
error mit Buch 13 auf Seite 1
error mit Buch 14 auf Seite 1
error mit Buch 15 auf Seite 1
error mit Buch 16 auf Seite 1
error mit Buch 17 auf Seite 1
error mit Buch 18 auf Seite 1
error mit Buch 19 auf Seite 1
error mit Buch 20 auf Seite 1
error mit Buch 21 auf Seite 1
error mit Buch 22 auf Seite 1
error mit Buch 23 auf Seite 1
error mit Buch 24 auf Seite 1
error mit Buch 25 auf Seite 1
error mit Buch 11 auf Seite 2
error mit Buch 12 auf Seite 2
error mit Buch 13 auf Seite 2
error mit Buch 14 auf Seite 2
error mit Buch 15 auf Seite 2
error mit Buch 16 auf Seite 2
error mit Buch 17 auf Seite 2
error mit Buch 18 auf Seite 2
error mit Buch 19 auf Seite 2
error mit Buch 20 auf Seite 2
error mit Buch 21 auf Seite 2
error mit Buch 22 auf Seite 2
error mit Buch 23 auf Seite 2
error mit Buch 24 auf Seite 2
error mit Buch 25 auf Seite 2
error mit Buch 12 auf Seite 3
error mit Buch 13 auf Seite 3
error mit 

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[2]/div/div/main/div[3]/div[2]/div[1]/div[5]/div/a[2]"}
  (Session info: chrome=113.0.5672.93)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00BC8893+48451]
	(No symbol) [0x00B5B8A1]
	(No symbol) [0x00A65058]
	(No symbol) [0x00A90467]
	(No symbol) [0x00A9069B]
	(No symbol) [0x00ABDD92]
	(No symbol) [0x00AAA304]
	(No symbol) [0x00ABC482]
	(No symbol) [0x00AAA0B6]
	(No symbol) [0x00A87E08]
	(No symbol) [0x00A88F2D]
	GetHandleVerifier [0x00E28E3A+2540266]
	GetHandleVerifier [0x00E68959+2801161]
	GetHandleVerifier [0x00E6295C+2776588]
	GetHandleVerifier [0x00C52280+612144]
	(No symbol) [0x00B64F6C]
	(No symbol) [0x00B611D8]
	(No symbol) [0x00B612BB]
	(No symbol) [0x00B54857]
	BaseThreadInitThunk [0x75977D59+25]
	RtlInitializeExceptionChain [0x7744B74B+107]
	RtlClearBits [0x7744B6CF+191]


## Datawrangling

In [18]:
buch_df.shape

(99, 16)

In [19]:
buch_df

,title,preis,untertitel,autor,verlag,sprache,einband,erscheinungsjahr,seiten,artikelnummer,isbn,auflage,reihe,reihenbandnummer,genre,text
0,Der Feind,nan,nan,"Brand, Christine",nan,Deutsch,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,Die Krume Brot,nan,nan,"Bärfuss, Lukas",nan,Deutsch,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,Der Magier im Kreml,nan,nan,"Da Empoli, Giuliano",nan,Deutsch,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,Elternabend,nan,nan,"Fitzek, Sebastian",nan,Deutsch,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
4,Troubadour,nan,nan,"Walker, Martin",nan,Deutsch,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,Die kleine Freude,nan,nan,nan,nan,Deutsch,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
95,Unkaputtbar: Mein erstes Wimmelbuch: Wir mache...,nan,nan,"Schumann, Sibylle",nan,Deutsch,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
96,Glücklich durch Frust,nan,nan,"Maas, Rüdiger",nan,Deutsch,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
97,Die Nachricht,nan,nan,"Knecht, Doris",nan,Deutsch,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


## Ausblick

nächste Schritte:

* anpassung der Unterachiedlichen Buchinformationen

Ausblick:

* Bücher in den nächsten 30 Tagen könnten auch gescrapet werden